<a href="https://colab.research.google.com/github/Andreas-Lukito/Stock_Sentiment_Analysis/blob/dev%2Fandreas/notebooks/04_albert-base-v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Albert Fine-Tune Model for Predicting Sentiment

## Install Libraries

In [7]:
! pip install contractions emoji gensim optuna torch matplotlib

## Iport Libraries

In [16]:
# Common Python Libraries
import numpy as np
import pandas as pd
import os
import sys
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import random

# Deep Learning Libraries
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import AlbertTokenizerFast, AlbertForSequenceClassification
from torch.optim import Adam

# Data Preprocessing
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from sklearn.preprocessing import OrdinalEncoder
## Download nltk dependencies
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')

# Model metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error, r2_score

from google.colab import drive
drive.mount('/content/drive')

project_path = "/content/drive/MyDrive/stock_news_sentiment_analysis"

# Add the path to the text preprocessor
sys.path.append(os.path.abspath(os.path.join(project_path, "lib")))

## Import preprocessor
from preprocessor import clean_text

# Project Seed for Reproducability
SEED = random.randint(0, 2**32 - 1)  # Random integer between 0 and 2^32-1
print(f"seed: {SEED}")

model_name = "albert-base-v2"

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
seed: 3624058130


## Choose Device

In [17]:
# Detect available device
if torch.cuda.is_available():
    # check if ROCm backend is active
    if torch.version.hip is not None:
        backend = "ROCm"
    else:
        backend = "CUDA"

    device = torch.device("cuda")
    print(f"PyTorch is using GPU: {torch.cuda.get_device_name(0)}")
    print(f"Backend: {backend}")
else:
    device = torch.device("cpu")
    print("PyTorch is not using GPU — running on CPU")

PyTorch is using GPU: NVIDIA A100-SXM4-80GB
Backend: CUDA


## Import Data

In [18]:
before_date = "2025-11"

# Data path
cleaned_data_path = os.path.join(project_path,f"news_cache/{before_date}/csv/")
clean_cached_file = os.path.join(cleaned_data_path, f"{before_date}_clean_news_data.csv")

# Import Data
news_data = pd.read_csv(filepath_or_buffer=clean_cached_file, sep=',')

In [19]:
news_data.head()

,uuid,title,description,keywords,snippet,url,image_url,language,published_at,source,relevance_score,entities,similar,sentiment,text,length,clean_text
0,487e6a88-d3c2-4ae1-8dc2-26af6b31d688,2025: The Year Of Alphabet (GOOG),No stock has seen a bigger jump recently than ...,NaN,vzphotos/iStock Editorial via Getty Images\n\n...,https://seekingalpha.com/article/4848680-2025-...,https://static.seekingalpha.com/cdn/s3/uploads...,en,2025-11-30T05:30:00.000000Z,seekingalpha.com,NaN,"[{'symbol': 'GOOGL', 'name': 'Alphabet Inc.', ...",[],0.0000,vzphotos/iStock Editorial via Getty Images\n\n...,42,vzphotos istock editorial via getty images sin...
1,92b5c2bd-d324-4ae8-b115-2cfd95a8fa98,Why I'm Doubling Down On My Adobe Position (NA...,"Adobe's revenue is highly predictable, driven ...",NaN,To say that Adobe ( ADBE ) stock has not had a...,https://seekingalpha.com/article/4848762-why-i...,https://static.seekingalpha.com/cdn/s3/uploads...,en,2025-11-30T05:25:01.000000Z,seekingalpha.com,NaN,"[{'symbol': 'ADBE', 'name': 'Adobe Inc.', 'exc...",[],0.0000,Why I'm Doubling Down On My Adobe Position (NA...,1,doubling adobe position nasdaq adbe adobe reve...
2,9084e5f1-75f5-4f15-aa3d-0676073b4aaf,Global week ahead: The start of a Santa Rally ...,NaN,"STOXX 600, business news",And just like that... December is upon us. It'...,https://www.cnbc.com/2025/11/30/global-week-ah...,https://image.cnbcfm.com/api/v1/image/10823257...,en,2025-11-30T05:10:58.000000Z,cnbc.com,NaN,"[{'symbol': 'M', 'name': ""Macy's, Inc."", 'exch...",[],0.6908,And just like that... December is upon us. It'...,493,like december upon us volatile handover novemb...
3,7d36a275-f3a3-44ea-8cbc-caa0d67749c4,Global Risk Monitor: Week in Review – Nov 28,KEY ISSUES Silver surged 13% for the week and ...,NaN,KEY ISSUES\n\nSilver surged 13% for the week a...,https://global-macro-monitor.com/2025/11/29/gl...,https://global-macro-monitor.com/wp-content/up...,en,2025-11-30T05:07:50.000000Z,global-macro-monitor.com,NaN,"[{'symbol': 'NVDA', 'name': 'NVIDIA Corporatio...",[],-0.3612,KEY ISSUES\n\nSilver surged 13% for the week a...,729,key issues silver surged 13 week 90 year date ...
4,42ba634c-b7ce-491a-91c0-e2b1424af827,"Mcap boost: 7 of top-10 firms gain ₹96,201 cr;...",Market valuations of seven top firms rose by ₹...,NaN,The combined market valuation of seven of the ...,https://www.thehindubusinessline.com/markets/m...,https://bl-i.thgim.com/public/incoming/ji6cih/...,en,2025-11-30T05:04:20.000000Z,thehindubusinessline.com,NaN,"[{'symbol': 'SBKFF', 'name': 'State Bank of In...",[],0.0000,The combined market valuation of seven of the ...,254,combined market valuation seven top 10 valued ...


## Split the data to Train, Test, and Validation

In [20]:
test_size = 0.20
val_size = 0.10

# Splitting the data into train and temp (which will be further split into validation and test)
train_df, test_df = train_test_split(news_data, test_size=test_size, random_state=SEED)

# Splitting train into validation and test sets
train_df, val_df = train_test_split(train_df, test_size=val_size, random_state=SEED)

In [40]:
x_train = train_df["clean_text"].tolist()
y_train = train_df["sentiment"].tolist()

x_test = test_df["clean_text"].tolist()
y_test = test_df["sentiment"].tolist()

x_val = val_df["clean_text"].tolist()
y_val = val_df["sentiment"].tolist()

## Data Preprocessing

### Tokenizer for the text

In [50]:
tokenizer = AlbertTokenizerFast.from_pretrained(model_name)

class sentiment_text(torch.utils.data.Dataset): # create a class that behaves like torch.utils.data.Dataset
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer( # converts raw text -> model input
                                    texts,
                                    truncation = True,
                                    padding = True,
                                    max_length = 256 # since the max length of the tweets are around 35 - 40 words
                                )

        # get the labels
        self.labels = labels

    def __getitem__(self, index): # so that pytorch can get the data (returns one sample of the data)
        item = {key: torch.tensor(val[index]) for key, val in self.encodings.items()} # self.encoding stores (input_ids, attention_mask, label)
        item["labels"] = torch.tensor(self.labels[index], dtype=torch.float32) # get the label on the chosen index while converting to a torch tensor format
        return item

    def __len__(self): #to get the length of the data (used when batching)
        return len(self.labels)

In [51]:
train_dataset = sentiment_text(x_train, y_train, tokenizer)
test_dataset  = sentiment_text(x_test, y_test, tokenizer)
val_dataset  = sentiment_text(x_val, y_val, tokenizer)

### Data Loader for the Model

In [63]:

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32)
val_loader  = DataLoader(val_dataset, batch_size=32)

## Train Model

In [53]:
model = AlbertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=1 # Since This is Regression
)

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
optimizer = Adam(model.parameters(), lr=5e-5)

In [66]:
model.to(device)
model.train() #make the model to training mode

for epoch in tqdm(range(10), desc="Training Albert Model", unit="epoch"):  # number of epochs

    for batch in train_loader:
        for k, v in batch.items():
            batch[k] = v.to(device)

        optimizer.zero_grad() # Resets all gradients to zero before computing new ones.
        outputs = model(**batch)  # forward pass
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} loss: {loss.item()}")

Training Albert Model:  10%|█         | 1/10 [01:01<09:11, 61.33s/epoch]

Epoch 1 loss: 0.05736361816525459


Training Albert Model:  20%|██        | 2/10 [02:02<08:10, 61.31s/epoch]

Epoch 2 loss: 0.08571282774209976


Training Albert Model:  30%|███       | 3/10 [03:03<07:09, 61.30s/epoch]

...

Epoch 8 loss: 0.007440296933054924


Training Albert Model:  90%|█████████ | 9/10 [09:11<01:01, 61.28s/epoch]

Epoch 9 loss: 0.00716694351285696


Training Albert Model: 100%|██████████| 10/10 [10:12<00:00, 61.29s/epoch]

Epoch 10 loss: 0.015587625093758106


## Model Evaluation

In [67]:
def evaluate_model(model, data_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in data_loader:
            # move batch to device
            for k, v in batch.items():
                batch[k] = v.to(device)

            # forward pass
            outputs = model(**batch) # turns raw inputs to named inputs as hugging face expects

            preds = outputs.logits.squeeze(-1)
            labels = batch["labels"].squeeze(-1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # metrics
    mse = mean_squared_error(all_labels, all_preds)
    mae = mean_absolute_error(all_labels, all_preds)
    rmse = root_mean_squared_error(all_labels, all_preds)
    r2 = r2_score(all_labels, all_preds)

    return mse, mae, rmse, r2


In [68]:
mse, mae, rmse, r2 = evaluate_model(
                                    model,
                                    test_loader,
                                    device
                                    )


print(f"mse       = {mse:.4f}")
print(f"mae       = {mae:.4f}")
print(f"rmse      = {rmse:.4f}")
print(f"r²        = {r2:.4f}")

mse       = 0.0888
mae       = 0.2182
rmse      = 0.2980
r²        = 0.0917
